In [1]:
using ITensors
using Plots
using Random
using Statistics
using LinearAlgebra


"""
I HAVENT TESTED THIS YET, THIS IS HERE TO PRODUCE GRAPH FIGURE 8 FROM THE REPORT


"""

"I HAVENT TESTED THIS YET, THIS IS HERE TO PRODUCE GRAPH FIGURE 8 FROM THE REPORT\n\n\n"

In [2]:
"""
Creates a random MPS for a spin-1/2 chain of length L.
The initial state is antiferromagnetic, which helps DMRG converge.
"""
function create_MPS(L::Int)
    sites = siteinds("S=1/2", L)
    # Start with an antiferromagnetic state "|↑↓↑↓...⟩"
    initial_state = [isodd(i) ? "Up" : "Dn" for i in 1:L]
    ψ₀ = randomMPS(sites, initial_state)
    return ψ₀, sites
end

create_MPS

In [3]:
"""
Creates a weighted adjacency matrix
"""
function create_weighted_adj_mat(N::Int, σ::Float64; μ::Float64=1.0)
    if σ == 0.0
        A = ones(Float64, N, N)
        A -= Matrix{Float64}(I, N, N)
        return A
    end

    A = zeros(Float64, N, N)
    for i in 1:N, j in (i+1):N
        weight = μ + σ * randn()
        A[i, j] = A[j, i] = weight
    end
    return A
end

create_weighted_adj_mat

In [4]:
"""
Creates a weighted adjacency matrix
"""
function create_weighted_adj_mat(N::Int, σ::Float64; μ::Float64=1.0)
    if σ == 0.0
        A = ones(Float64, N, N)
        A -= Matrix{Float64}(I, N, N)
        return A
    end

    A = zeros(Float64, N, N)
    for i in 1:N, j in (i+1):N
        weight = μ + σ * randn()
        A[i, j] = A[j, i] = weight
    end
    return A
end

create_weighted_adj_mat

In [5]:
"""
Creates the MPO for the XXZ Hamiltonian on a graph with weighted interactions.
"""
function create_weighted_xxz_mpo(N::Int, adj_mat, sites; J::Float64=1.0, Δ::Float64=1.0)
    ampo = OpSum()
    for i in 1:N-1
        for j in i+1:N
            coupling_strength = adj_mat[i, j]
            if coupling_strength != 0.0
                ampo += coupling_strength * (J / 2), "S+", i, "S-", j
                ampo += coupling_strength * (J / 2), "S-", i, "S+", j
                ampo += coupling_strength * (J * Δ), "Sz", i, "Sz", j
            end
        end
    end
    return MPO(ampo, sites)
end

create_weighted_xxz_mpo

In [6]:

function plot_bond_dim()

    N_range = 10:1:75       # System sizes from 10 to 75 nodes
    num_sweeps = 30         # Number of DMRG sweeps, as per the report
    max_bond_dim = 200      # A high limit for the bond dimension during DMRG
    cutoff = 1E-10          # Truncation error cutoff for DMRG

    bond_dims = Float64[]

    for N in N_range

        ψ₀, sites = create_MPS(N)
        adj_mat = create_weighted_adj_mat(N, 0.0) # Homogeneous graph
        H_mpo = create_weighted_xxz_mpo(N, adj_mat, sites; J=1.0, Δ=1.5)

        sweeps = Sweeps(num_sweeps)
        setmaxdim!(sweeps, max_bond_dim)
        setcutoff!(sweeps, cutoff)

        # obs used to find max bond dim after last sweep
        obs = DMRGObserver(["MaxDim"])
        energy, ψ_gs = dmrg(H_mpo, ψ₀, sweeps; observer=obs, outputlevel=0)
        
        # maximum bond dimension of the final ground state MPS
        final_bond_dim = maxlinkdim(ψ_gs)
        push!(bond_dims, final_bond_dim)

        println("Completed N = $N. Energy = $(round(energy, digits=4)), Max Bond Dim = $final_bond_dim")
    end

    plt = plot(
        title="Saturated Bond Dimension for an Average Graph with N Nodes",
        xlabel="Number of Nodes",
        ylabel="Average Bond Dimension Required",
        legend=false,
        gridalpha=0.3,
        framestyle=:box
    )

    plot!(plt, N_range, bond_dims,
        lw=2,
        marker=:circle,
        markersize=4,
        markerstrokecolor=:auto,
        markercolor=:coral,
        linecolor=:deepskyblue
    )
    
    ylims!(plt, floor(Int, minimum(bond_dims))-1, ceil(Int, maximum(bond_dims))+1)

    return plt
end



plot_bond_dim (generic function with 1 method)

In [7]:
plt = plot_bond_dim();
display(plt)

LoadError: UndefVarError: `siteinds` not defined in `Main`
Suggestion: check for spelling errors or missing imports.